In [1]:
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import string
from nltk.stem import WordNetLemmatizer
from datetime import timedelta
import re


First we read in the tweets data of the three month

In [2]:
aug_tweet = pd.read_excel('august.xlsx', parse_dates = ['CreatedTime'])
sep_tweet = pd.read_excel('september.xlsx', parse_dates = ['CreatedTime'])
oct_tweet = pd.read_excel('october.xlsx', parse_dates = ['CreatedTime'])

Check `RangeIndex` then you know how many tweets are posted in each month

In [13]:
print(type(aug_tweet.info()))
print('=========================================')
print(type(sep_tweet.info()))
print('=========================================')
print(type(oct_tweet.info()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2309 entries, 0 to 2308
Data columns (total 48 columns):
UniversalMessageId        2309 non-null object
SocialNetwork             2309 non-null object
SenderUserId              2309 non-null int64
SenderScreenName          2309 non-null object
SenderListedName          2309 non-null object
SenderProfileImgUrl       2309 non-null object
SenderProfileLink         0 non-null float64
Sender Followers Count    2309 non-null int64
SenderInfluencerScore     0 non-null float64
SenderAge                 0 non-null float64
SenderGender              162 non-null object
Title                     0 non-null float64
Message                   2309 non-null object
MessageType               2309 non-null object
CreatedTime               2309 non-null datetime64[ns]
Language                  2309 non-null object
LanguageCode              2309 non-null object
CountryCode               1644 non-null object
MediaTypeList             1139 non-null object
Per

Still this time, we use key word to different relevant and irrelevant tweets. Add new found words in the below list, so the algorithm can use the updated one. **One more issue, for some retweet messages, FH related information is not contained in the message part, and this will generate some errors in the following messages**

In [114]:
none_FH_key = ['auto', 'car', 'game', 'field', 'happy', 'photo', 'sport', 'Match', 'LIVE', 'ford', 'picture', 'team',
               'pic', 'play', 'score', 'UCW', 'FinalHour', 'Win', 'youtube', 'youtu.be']
FH_key_dataframe = pd.read_csv('fh_keyword.csv')
FH_key = FH_key_dataframe['FH_key']

In [115]:
print(none_FH_key)

['auto', 'car', 'game', 'field', 'happy', 'photo', 'sport', 'Match', 'LIVE', 'ford', 'picture', 'team', 'pic', 'play', 'score', 'UCW', 'FinalHour', 'Win', 'youtube', 'youtu.be']


In [116]:
print(FH_key)

0                            knowFH
1                           Know FH
2                        FHcantwait
3                           Race4FH
4                       FHAwareness
5                      FH Awareness
6                    FHAwarenessDay
7                            FindFH
8                              HeFH
9                    HeterozygousFH
10                             HoFH
11                     HomozygousFH
12        Cholesteroleducationmonth
13        Cholesterolawarenessmonth
14     FamilialHypercholesterolemia
15                     FH Awareness
16                 FH Awareness Day
17                             HeFH
18                  Heterozygous FH
19                             HoFH
20                    Homozygous FH
21      Cholesterol education month
22      Cholesterol awareness month
23    Familial Hypercholesterolemia
24                    FH Foundation
25                        FHPatient
26                          Patient
27                        Ho

In [147]:
def relevant(msg):
    r = any(i.lower() in msg.lower() for i in FH_key)
    ir = any(i.lower() in msg.lower() for i in none_FH_key)    
    return not (not r and ir)


aug_tweet['relevance'] = aug_tweet['Message'].apply(relevant)
sep_tweet['relevance'] = sep_tweet['Message'].apply(relevant)
oct_tweet['relevance'] = oct_tweet['Message'].apply(relevant)

Now we give the relevant/irrelevant tweets number for each month, from which you can compute the portion of the no. of genuine tweets

In [148]:
print('Aug tweets')
print(aug_tweet.groupby('relevance')['relevance'].count())
print('========================')
print('Sep tweets')
print(sep_tweet.groupby('relevance')['relevance'].count())
print('========================')
print('Oct tweets')
print(oct_tweet.groupby('relevance')['relevance'].count())

Aug tweets
relevance
False     295
True     2014
Name: relevance, dtype: int64
Sep tweets
relevance
False     195
True     5453
Name: relevance, dtype: int64
Oct tweets
relevance
False     242
True     2101
Name: relevance, dtype: int64


Now we split the relevant tweets by posters. Still, we use individual poster names

In [6]:
import spacy

We extract the relevant tweets for each month

In [122]:
aug_tweet_true = aug_tweet[aug_tweet.relevance].reset_index(drop = True)
aug_tweet_false = aug_tweet[aug_tweet.relevance == False].reset_index(drop = True)
sep_tweet_true = sep_tweet[sep_tweet.relevance].reset_index(drop = True)
sep_tweet_false = sep_tweet[sep_tweet.relevance == False].reset_index(drop = True)
oct_tweet_true = oct_tweet[oct_tweet.relevance].reset_index(drop = True)
oct_tweet_false = oct_tweet[oct_tweet.relevance == False].reset_index(drop = True)

In [123]:
relevant_tweet = [aug_tweet_true, sep_tweet_true, oct_tweet_true]
months = ['aug', 'sep', 'oct']
for i, month in zip(relevant_tweet, months):
    i.to_csv(month+'TRUE.csv', index=False)
irrelevant_tweet = [aug_tweet_false, sep_tweet_false, oct_tweet_false]
for i, month in zip(irrelevant_tweet, months):
    i.to_csv(month+'FALSE.csv', index=False)

In [124]:
nlp = spacy.load('en_core_web_sm')
def INDorORG(name):
    name = re.sub(r'[^a-zA-Z\s]', '', name)
    doc = nlp(name)
    return [(ent.label_, ent.text) for ent in doc.ents]

In [129]:
aug_ner = aug_tweet_true['SenderListedName'].apply(INDorORG) ### attempt to use spacy.nlp for automatic Ind/Org split
sep_ner = sep_tweet_true['SenderListedName'].apply(INDorORG)
oct_ner = oct_tweet_true['SenderListedName'].apply(INDorORG)

In [131]:
print(aug_ner.apply(lambda x: len(x)==0).sum())
print(aug_ner.size)
print(sep_ner.apply(lambda x: len(x)==0).sum())
print(sep_ner.size)
print(oct_ner.apply(lambda x: len(x)==0).sum())
print(oct_ner.size)

526
1919
1485
5312
487
1975


We need to use the word again.

In [132]:
org_keyword = ['foundation', 'foundtion[sic.]', 'health', 'department', 'organization', 'agency', 'news', 'group',
               'society', 'committee', 'volunteer', 'county', 'government', 'network', 'firm', 'company', 'companies',
               'blog', 'we', 'promotions', 'marketing', 'forum', 'consult', 'national', 'cardio', 'AHA', 'international',
               'association', 'heart', 'America', 'campaign', 'pharmacy', 'pharmaceutical', 'Gene', 'Society', 'gene',
               'genom', 'genetic', 'medicine', 'lab']

In [133]:
def isORG(name):
    name = re.sub(r'[^a-zA-Z\s]', '', name)    
    return any(i.lower() in name.lower() for i in org_keyword)

In [134]:
aug_tweet_true['isORG'] = aug_tweet_true['SenderListedName'].apply(isORG) ### attempt to use spacy.nlp for automatic Ind/Org split
sep_tweet_true['isORG']  = sep_tweet_true['SenderListedName'].apply(isORG)
oct_tweet_true['isORG']  = oct_tweet_true['SenderListedName'].apply(isORG)

Now we can split tweets w.r.t the identity of posters

In [135]:
aug_tweet_true_org = aug_tweet_true[aug_tweet_true['isORG'] == True].reset_index(drop=True)
sep_tweet_true_org = sep_tweet_true[sep_tweet_true['isORG'] == True].reset_index(drop=True)
oct_tweet_true_org = oct_tweet_true[oct_tweet_true['isORG'] == True].reset_index(drop=True)

aug_tweet_true_ind = aug_tweet_true[aug_tweet_true['isORG'] == False].reset_index(drop=True)
sep_tweet_true_ind = sep_tweet_true[sep_tweet_true['isORG'] == False].reset_index(drop=True)
oct_tweet_true_ind = oct_tweet_true[oct_tweet_true['isORG'] == False].reset_index(drop=True)

In [136]:
#import bpy
org = [aug_tweet_true_org, sep_tweet_true_org, oct_tweet_true_org]
months = ['aug', 'sep', 'oct']
for i, month in zip(org, months):
    i.to_csv(month+'org.csv', index=False)
ind = [aug_tweet_true_ind, sep_tweet_true_ind, oct_tweet_true_ind]
for i, month in zip(ind, months):
    i.to_csv(month+'individual.csv', index=False)

We count unique posters for each month

In [165]:
print('aug unique org:', aug_tweet_true_org['SenderListedName'].unique().size)
print('aug unique ind:', aug_tweet_true_ind['SenderListedName'].unique().size)
print('sep unique org:', sep_tweet_true_org['SenderListedName'].unique().size)
print('sep unique ind:', sep_tweet_true_ind['SenderListedName'].unique().size)
print('oct unique org:', oct_tweet_true_org['SenderListedName'].unique().size)
print('oct unique ind:', oct_tweet_true_ind['SenderListedName'].unique().size)

aug unique org: 76
aug unique ind: 681
sep unique org: 315
sep unique ind: 1329
oct unique org: 76
oct unique ind: 614


The table for message type 

In [156]:
print('Aug org message type')
print(aug_tweet_true_org.groupby('MessageType')['MessageType'].count())
print('=========================')
print('Aug ind message type')
print(aug_tweet_true_ind.groupby('MessageType')['MessageType'].count())
print('=========================')
print('Sep org message type')
print(sep_tweet_true_org.groupby('MessageType')['MessageType'].count())
print('=========================')
print('Sep ind message type')
print(sep_tweet_true_ind.groupby('MessageType')['MessageType'].count())
print('=========================')
print('Oct org message type')
print(oct_tweet_true_org.groupby('MessageType')['MessageType'].count())
print('=========================')
print('Oct ind message type')
print(oct_tweet_true_ind.groupby('MessageType')['MessageType'].count())

Aug org message type
MessageType
Twitter Mention     67
Twitter Reply       12
Twitter Retweet     80
Twitter Update     114
Name: MessageType, dtype: int64
Aug ind message type
MessageType
Twitter Mention     178
Twitter Reply        84
Twitter Retweet    1114
Twitter Update      270
Name: MessageType, dtype: int64
Sep org message type
MessageType
Twitter Mention    258
Twitter Reply        8
Twitter Retweet    489
Twitter Update     437
Name: MessageType, dtype: int64
Sep ind message type
MessageType
Twitter Mention     467
Twitter Reply        99
Twitter Retweet    2814
Twitter Update      740
Name: MessageType, dtype: int64
Oct org message type
MessageType
Twitter Mention     72
Twitter Reply        2
Twitter Retweet    124
Twitter Update      97
Name: MessageType, dtype: int64
Oct ind message type
MessageType
Twitter Mention    391
Twitter Reply       58
Twitter Retweet    926
Twitter Update     305
Name: MessageType, dtype: int64


In [ ]:
aug_org_group = aug_tweet_true_org.groupby('MessageType')
sep_org_group = sep_tweet_true_org.groupby('MessageType')
oct_org_group = oct_tweet_true_org.groupby('MessageType')

In [157]:
writer = pd.ExcelWriter('aug_org_type.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in aug_org_group.groups.keys():
    aug_org_group.get_group(i).to_excel(writer, sheet_name=i)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

writer = pd.ExcelWriter('sep_org_type.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in sep_org_group.groups.keys():
    sep_org_group.get_group(i).to_excel(writer, sheet_name=i)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

writer = pd.ExcelWriter('oct_org_type.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in oct_org_group.groups.keys():
    oct_org_group.get_group(i).to_excel(writer, sheet_name=i)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [159]:
aug_ind_group = aug_tweet_true_ind.groupby('MessageType')
sep_ind_group = sep_tweet_true_ind.groupby('MessageType')
oct_ind_group = oct_tweet_true_ind.groupby('MessageType')

In [160]:
writer = pd.ExcelWriter('aug_ind_type.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in aug_ind_group.groups.keys():
    aug_ind_group.get_group(i).to_excel(writer, sheet_name=i)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

writer = pd.ExcelWriter('sep_ind_type.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in sep_ind_group.groups.keys():
    sep_ind_group.get_group(i).to_excel(writer, sheet_name=i)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

writer = pd.ExcelWriter('oct_ind_type.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in oct_ind_group.groups.keys():
    oct_ind_group.get_group(i).to_excel(writer, sheet_name=i)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [170]:
org_topic_keyword = pd.read_excel('6 categories for organizations.xlsx')

In [171]:
print(org_topic_keyword)

   Awareness/Prevention Knowledge Dissemination/Research  \
0              National                          podcast   
1         International                          journal   
2                KnowFH                         research   
3             Education                           inform   
4             Awareness                             blog   
5            FHcantwait                        resources   
6           cholesterol                            paper   
7                 learn                        questions   
8           tweethathon                        published   
9               prevent                          webinar   
10              cascade                      publication   
11              genetic                            video   
12            screening                          authors   
13            detection                              NaN   
14             campaign                              NaN   
15               FindFH                 

In [172]:
Stop = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

In [176]:
def topicmatch(msg, topic_keyword_dataframe):
    topic = []
    msg = re.sub(r'[^\w]', ' ', msg)
    #token_msg = word_tokenize(msg)
    #is_word = list(map(lambda x: x not in Stop, token_msg))
    #lemma_token_msg = [wordnet_lemmatizer.lemmatize(word) for word in token_msg[is_word]]
    for category in topic_keyword_dataframe:
        key_word = topic_keyword_dataframe[category].dropna()
        if any(i.lower() in msg.lower() for i in key_word):
            topic.append(category)
            
    return topic
    
aug_tweet_org_topic = aug_tweet_true_org['Message'].apply(topicmatch, topic_keyword_dataframe=org_topic_keyword)
sep_tweet_org_topic = sep_tweet_true_org['Message'].apply(topicmatch, topic_keyword_dataframe=org_topic_keyword)
oct_tweet_org_topic = oct_tweet_true_org['Message'].apply(topicmatch, topic_keyword_dataframe=org_topic_keyword)

In [177]:
import itertools
aug_org_topic_count = Counter(list(itertools.chain.from_iterable(aug_tweet_org_topic)))
sep_org_topic_count = Counter(list(itertools.chain.from_iterable(sep_tweet_org_topic)))
oct_org_topic_count = Counter(list(itertools.chain.from_iterable(oct_tweet_org_topic)))

In [178]:
print('topic for aug org:', aug_org_topic_count)
print('topic for sep org:', sep_org_topic_count)
print('topic for oct org:', oct_org_topic_count)

topic for aug org: Counter({'Awareness/Prevention': 262, 'Diagnosis/Symptoms': 167, 'Knowledge Dissemination/Research': 48, 'Fundraiser/Conferences/Summits': 35, 'Treatment': 35, 'Lifestyle': 3})
topic for sep org: Counter({'Awareness/Prevention': 1177, 'Diagnosis/Symptoms': 683, 'Knowledge Dissemination/Research': 201, 'Treatment': 135, 'Lifestyle': 109, 'Fundraiser/Conferences/Summits': 71})
topic for oct org: Counter({'Awareness/Prevention': 278, 'Diagnosis/Symptoms': 142, 'Fundraiser/Conferences/Summits': 114, 'Knowledge Dissemination/Research': 75, 'Treatment': 57, 'Lifestyle': 25})


In [179]:
ind_topic_keyword = pd.read_excel('categories for individuals.xlsx')

In [180]:
aug_tweet_ind_topic = aug_tweet_true_ind['Message'].apply(topicmatch, topic_keyword_dataframe=ind_topic_keyword)
sep_tweet_ind_topic = sep_tweet_true_ind['Message'].apply(topicmatch, topic_keyword_dataframe=ind_topic_keyword)
oct_tweet_ind_topic = oct_tweet_true_ind['Message'].apply(topicmatch, topic_keyword_dataframe=ind_topic_keyword)

aug_ind_topic_count = Counter(list(itertools.chain.from_iterable(aug_tweet_ind_topic)))
sep_ind_topic_count = Counter(list(itertools.chain.from_iterable(sep_tweet_ind_topic)))
oct_ind_topic_count = Counter(list(itertools.chain.from_iterable(oct_tweet_ind_topic)))

print('topic for aug ind:', aug_ind_topic_count)
print('topic for sep ind:', sep_ind_topic_count)
print('topic for oct ind:', oct_ind_topic_count)

topic for aug ind: Counter({'Awareness/Prevention': 1354, 'Diagnosis/Symptoms': 1048, 'Knowledge Dissemination/Research': 398, 'Treatment': 233, 'Fundraiser/Conferences/Summits': 188, 'Lifestyle': 72})
topic for sep ind: Counter({'Awareness/Prevention': 3781, 'Diagnosis/Symptoms': 2532, 'Knowledge Dissemination/Research': 674, 'Treatment': 473, 'Lifestyle': 324, 'Fundraiser/Conferences/Summits': 300})
topic for oct ind: Counter({'Awareness/Prevention': 1315, 'Diagnosis/Symptoms': 828, 'Fundraiser/Conferences/Summits': 699, 'Knowledge Dissemination/Research': 358, 'Treatment': 304, 'Lifestyle': 57})
